In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%reload_ext autoreload

In [ ]:
%alias nb_convert ~/bin/develtools/nbconvert createFolders.ipynb

In [ ]:
%nb_convert

In [3]:
import constants
# import logging first to prevent any sub modules from creating the root logger
import logging
from logging import handlers
from logging import config
logging.config.fileConfig(constants.logging_config, defaults={'logfile': constants.log_file})



In [4]:
import csv
import sys
from pathlib import Path
import subprocess
import time
import ArgConfigParse

In [5]:
def csv_to_list(file):
    '''read csv file `file` into a list
    
    Guess the CSV dialect (e.g. tsv, csv, etc.)
    
    Returns `list`'''
    logging.debug(f'reading {file} to list')
    csvFile = Path(file).expanduser().absolute()
    file_csv = []
    # try to figure out the dialect (csv, tsv, etc.)
    with open(csvFile, 'r') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        file.seek(0)
        reader = csv.reader(file, dialect)
        for row in reader:
            file_csv.append(row)

    return file_csv

In [6]:
def map_headers(csv_list, expected_headers=[]):
    '''map row 0 of a csv as formatted as a list to a dictionary of expected header values'''
    missing_headers = []
    header_map = {}
    
    csvHeader = csv_list[0]
    logging.debug('mapping headers')
    logging.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
            
    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
        
    logging.debug('completed mapping')
    return(header_map, missing_headers)

In [7]:
def do_exit(e='unknown error in unknown module: BSoD!', exit_status=0, testing=False):

        
    print('\n'*4)
    if exit_status > 0:
        logging.error(f'exited before completion with exit code {exit_status}')
        logging.error(e)
    print(e)
    if not testing:
        try:
            sys.exit(exit_status)
        except SystemExit:
            pass

In [8]:
class gd_path():
    def __init__(self, path=None):
        '''google drive path class
        
        Attributes:
            path(`str`): path to google drive drive object'''
        self.confirmed = False
        self.path = path
        self._file_base = 'https://drive.google.com/file/d/'
        self._dir_base = 'https://drive.google.com/drive/folders/'
        self.is_file = False   
    
    
    def __repr__(self):
        return f'gd_path({self.path})'
    
    def __str__(self):
        return f'{self.path}'
    
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path):
        '''full path to object
        
        Args:
            path(`str` or `Path`): /path/to/object
            
        Sets Attributes:
            self.path: path to object
            self.root: same as path for directories, parent directory for files
            self.is_file: true for files and file-like objects, false for directories'''
        if not path:
            self._path = None
        else:
            self._path = Path(path)
            if self._path.is_dir() and self._path.exists():
                self.root = self._path
                self.is_file = False
            if self.path.is_file() and self._path.exists():
                self.root = self._path.parent
                self.is_file = True
            
            if not self._path.exists():
                self.is_file = False
                self.root = self._path.parent

    @property
    def webview_link(self):
        '''full webview link to object in google drive'''
        self._webview_link = None
        try:
            item_id = self.get_xattr('user.drive.id')
        except FileNotFoundError as e:
            logging.debug(f'{e}')
            return None
        except ChildProcessError as e:
            logging.debug(f'{e}')
            return None

        if len(item_id) < 1:
            return None
        else:
            item_id = item_id[0]
        
        
        if not self.is_file:
            self._webview_link = f'{self._dir_base}{item_id}'
        if self.is_file:
            self._webview_link = f'{self._file_base}{item_id}'
        return self._webview_link
            
    def check_parent(self, expected):
        '''checks if the parent matches the expected parent'''
        if self.root.parents[0].name == expected:
            return True
        else:
            return False
        
    def get_xattr(self, attribute, file=None):
        '''get the extended attributes of a file or directory
        Args:
            file(`str` or Path): path to file
            attribute('`str`'): attribute key to access

        Returns:
            `list` - attribute or key: attribute pairs

        Raises:
            FileNotFoundError - file or directory does not exist
            ChildProcessError - xattr utility exits with non-zero code 
                This is common for files that have no extended attributes or do not
                have the requested attribute'''
        if not file:
            file = self.path
        else:
            file = Path(file).absolute()
            
        attributes = []
        if not file.exists():
            raise FileNotFoundError(file)

        p = subprocess.Popen(f'xattr -p  {attribute} "{file.resolve()}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in p.stdout.readlines():
            attributes.append(line.decode("utf-8").strip())
    #         attributes = attributes + line.decode("utf-8").strip()
        retval = p.wait()
        if retval != 0:
            raise ChildProcessError(f'xattr exited with value: {retval}')
        return attributes     

    @property
    def file_id(self, path=None):
        '''unique file id for each object (directories or file)
        
        Args:
            path(`str` or `Path`): path to object; defaults to self.path
        
        Returns:
            `list` of `str` containing the file id'''
        if not path:
            path = self.path
        try:
            file_id = self.get_xattr('user.drive.id', path)
        except FileNotFoundError as e:
            logging.info(f'\'{path}\' does not appear to exist; cannot get attributes')
            file_id = None
        return file_id
    
    def confirm(self, path=None):
        '''confirm that a created object has been sent over file stream
        
        Args:
            path(`str` or `Path`): path to object; default is self.path
        
        Returns:
            `list` of `str` containing the file id
            
        Attributes Set:
            self.confirmed: True when object has been sent'''
        
        if not path:
            path = self.path
        file_id = self.file_id
        
        if file_id:
            if 'local-' in file_id[0]:
                self.confirmed = False
                file_id = None
            else:
                self.confirmed = True
        return file_id
    
    def mkdir(self, path=None, parents=False, exist_ok=False, kwargs={}):
        '''create a directory using pathlib.Path().mkdir()
        
        Args:
            path(`str` or `Path`): path to create
            parents(`bool`): create parent directories - default false
            exists_ok(`bool`): do not raise error if directory exists
            kwargs: kwargs for pathlib.Path().mkdir()
            
        Returns:
            file_id(`list`)'''
        if not path:
            path = self.path
            logging.debug(f'using self.path: {path}')
        else:
            logging.debug(f'using supplied path: {path}')
            
        if path.is_file():
            raise TypeError(f'{path} is a file')
            
        path = Path(path)
            
        path.mkdir(parents=parents, exist_ok=exist_ok, **kwargs)
        if self.confirm(path):
            file_id = self.get_xattr('user.drive.id', path)
        return self.file_id
    

In [9]:
class student_path(gd_path):
    def __init__(self, path=None, class_of=None, id_number=None, name=None):
        '''student directory in google drive; child class of gd_path:
        
        Args:
        
        Properties:
            class_of(`str`): "ClassOf-YYYY" string representation of projected graduation year
            name(`str`): "Last, First" string representation of student name
            id_number(`int`): student id number
            matches(`dict`):  name and webview link of directories that contain "id_number"
            path_parts(`dict`): path compontents stored as dictionary keys'''
        
        super(student_path, self).__init__(path=path)
        self.matches = {}
        self.path_parts = {'ClassOf': None, 'id_number': None, 'name': None}
        self.class_of = class_of
        self.name = name
        self.id_number = id_number
    
    
    def __repr__(self):
        return f'student_path({self.student_dir_name})'
        
    def __str__(self):
        return f'{self.student_dir_name}'
    
    def get_xattr(self, attribute, file=None):
        if not file:
            file = self.student_dir_name
        return super().get_xattr(attribute, file)
    
    @property
    def class_of(self):
        return self._class_of
    
    @class_of.setter
    def class_of(self, class_of):
        '''string representation of projected graduation date in format: "ClassOf-YYYY"
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not class_of:
            self._class_of = None
        else:
            # attempt to coerce strings from cSV file into type int
            class_of = int(class_of)
            if not isinstance(class_of, int):
                raise TypeError('class_of must be of type `int`')
        self.path_parts['ClassOf'] = f'ClassOf-{class_of}'
        self._class_of = class_of
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        '''string representation of "Last, First" names
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not name:
            self._name = None
        else:
            if not isinstance (name, str):
                raise TypeError('name must be of type `str`')
        self.path_parts['name'] = name
        self._name = name
        
    @property
    def id_number(self):
        return self._id_number
    
    @id_number.setter
    def id_number(self, number):
        '''integer of student id number
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not number:
            self._id_number = None
        else:
            # try to coerce number into type int
            number = int(number)
            if not isinstance (number, int):
                raise TypeError('id_number must be of type `int`')
        self.path_parts['id_number'] = number
        self._id_number = number

    @property
    def student_dir_name(self):
        '''full absolute path to student directory in format:
            ClassOf-YYYY/Last, First - NNNNNNN'''
        d = f"/{self.path_parts['ClassOf']}/{self.path_parts['name']} - {self.path_parts['id_number']}"
        if self.path:
            # not sure why this is needed, but any joining of self.root/Path(d) fails
            d = f'{str(self.path)}/{d}'
        return Path(d)
    
    # method for checking for similarly named student folders in this ClassOf folder
    def check_similar(self):
        '''check for similarly named directories based on student id number 
        within the path/ClassOf/ directory
        
        Properties Set:
            self.matches(`dict`): dictionary of similar directories
        Returns:
            `bool`: True if matching directories found'''
        similar = False
        matches = {}
        for i in self.student_dir_name.parent.glob(f"*{self.path_parts['id_number']}*"):
            match_id = self.get_xattr('user.drive.id', self.student_dir_name.parent/i)
            if i.absolute().is_dir():
                url = '/'.join((self._dir_base, match_id[0]))
            else:
                url = '/'.join((self._file_base, match_id[0]))
            matches[str(i)] = url
        self.matches = matches
        if matches:
            similar = True
        return similar

    def mkdir(self, path=None, exist_ok=False, parents=True, kwargs={}):
        '''make a google drive directory using pathlib.Path().mkdir()
        
        Args:
            path(`str` or `Path`): defaults to self.student_dir_name
            exist_ok(`bool`): True - do not raise error if directory exists
            parents(`bool`): True - create parents if they do not exist
            kwargs({}): pathlib.Path() kwargs
            
        Returns:
            list[str]: google drive object ID string'''
        if not path:
            path = self.student_dir_name
        logging.debug(f'calling super().mkdir(path={path})')
        val = super().mkdir(path=path, exist_ok=exist_ok, parents=parents, **kwargs)
        return val
            
        

In [10]:
def validate_data(csv_list, expected_headers, header_map):
    '''validate list items for proper data types
         naievely attempts to coerce strings from CSV into expected_header types
         returns a tuple of list of rows that were successfully coerced and those
         that could not be coerced
    
    Args:
        csv_list(`list` of `list`): csv as a nested list [['h1', 'h2'], ['item', 'item2']]
        expected_headers(`dict`): {'literal_header': type} {'ClassOf':, int, 'Name', str}
        header_map(`dict`): map of list index for each header {'h1': 0, 'h2': 5, 'hN': i}
        
    Returns:
        (`tuple` of `list`): (valid_rows, invalid_rows)
    '''
    valid = []
    invalid = []

    for row in csv_list[1:]:
        good_row = True
        for k in expected_headers.keys():
            # test for coercable types
            try:
                test = expected_headers[k](row[header_map[k]])
            except ValueError:
#                 do_exit(f'Bad student.export: {k} contained {row[header_map[k]]}\ncannot continue. Please try running the export again.')
                logging.warning(f'{row}')
                logging.warning(f'Bad student.export: column "{k}" contained "{row[header_map[k]]}"--this should be {(expected_headers[k])}')
                invalid.append(row)
                good_row = False
                break
        if  good_row:
            valid.append(row)
        
    return valid, invalid
    

In [12]:
# def writeCSV(studentFolders, csvHeaders, output_path):
#     logger = logging.getLogger(__name__)
#     logger.debug('writing csv output at path: {}'.format(output_path))
    
#     output_path = os.path.expanduser(output_path)
#     htmlFormat = '<a href={}>Right click link and *Open Link in New Tab* to view student folder</a>'
#     csvOutput_list = []
#     if not csvHeaders:
#         csvHeaders = ['webViewLink',
#                       'LastFirst',
#                       'classOf', 
#                       'student_number']
#     csvOutput_list.append(csvHeaders)
#     for student in studentFolders:
#         if studentFolders[student]:
#             thisStudent = []
#             for header in csvHeaders:
#                 if header in studentFolders[student]:
#                     if header == 'webViewLink':
#                         thisStudent.append(htmlFormat.format(studentFolders[student][header]))
#                     else:
#                         thisStudent.append(studentFolders[student][header])                
#                         if len(thisStudent) == len(csvHeaders):
#                             csvOutput_list.append(thisStudent)
    
#     logger.debug('Writing rows:')
#     try:
# #         with open(output_path, 'wb') as f:
#         with open(output_path, 'w') as f:
#             writer = csv.writer(f,
#                     quoting = csv.QUOTE_NONE,
#                     delimiter='\t')
#             for each in csvOutput_list:
#                 logging.debug(each)
#                 writer.writerow(each)
# #             writer.writerows(csvOutput_list)
#     except Exception as e:
#         logger.error('error writing CSV file: {}; {}'.format(output_path, e))
#         return(None)
#     return(output_path)

In [27]:
def adjust_handler(handler=None, new_level=None):
    '''adjust a logging handler
    
    Args:
        handler(`str`): partial string in handler name - if none, returns list of all handlers attached to root
            '*' adjusts all handlers to new_level
        new_level(`str`): DEBUG, INFO, WARNING, ERROR
    
    Returns:
        `list`: list of handlers and levels currently set'''
    if not handler:
        return(logging.getLogger().handlers)
    
    my_handler = None    
    for index, val in enumerate(logging.getLogger().handlers):
        if handler == '*':
            my_handler = logging.getLogger().handlers[index]
        else:
            if handler in str(val):
                my_handler = logging.getLogger().handlers[index]
        if my_handler:
            logging.info(f'setting {str(my_handler)} to {new_level}')
            my_handler.setLevel(new_level)
        else:
            logging.warning(f'handler: "{handler}" not found')
        
    return logging.getLogger().handlers

In [51]:
def main():    
    # gather command line arguments
    logger = logging.getLogger(__name__)
    
    # adjust the root handler info - this will limit the logging from modules and other local functions
#     logger.root.setLevel('INFO')

    logger.debug('starting up...')
    args = ArgConfigParse.CmdArgs()
    args.add_argument('-s', '--student_export', ignore_none=False, metavar='/path/to/student.export.csv', 
                      type=str, dest='student_export', help='Export from PowerSchool containing: LastFirst, ClassOf, Student_Number')

    args.add_argument('-g', '--google_drive', ignore_none=True, metavar='/Volumes/GoogleDrive/Shared drives/ASH Cum Folders/folder/',
                      type=str, dest='main__drive_path', help='Full path to Google Drive Shared Drive containing cumulative files')

    args.add_argument('-l', '--log_level', ignore_none=True, metavar='ERROR, WARNING, INFO, DEBUG', 
                      type=str, dest='main__log_level', help='Logging level -- Default: WARNING')

    args.parse_args()
                      
    # create a parser for the config files
    config_file = Path(constants.config_file)
    user_config_path = Path(constants.user_config_path)
    
    # if the user configuration file is missing set to True & create later
#     if not user_config_path.exists:
#         update_user_config = True
#     else:
#         update_user_config = False                      
    update_user_config = user_config_path.exists
    logging.debug(f'update_user_config: {update_user_config}')
    
    # parse the config files
    parser = ArgConfigParse.ConfigFile(config_files=[config_file, user_config_path], ignore_missing=True)
    parser.parse_config()

    # merge the command line arguments and the config files
    config = ArgConfigParse.merge_dict(parser.config_dict, args.nested_opts_dict)

    # adjust the logging levels if needed
#     if args.opts_dict['main__log_level'] in (['DEBUG', 'INFO', 'WARNING', 'ERROR']):
    if config['main']['log_level']:
        ll = config['main']['log_level']
        if ll in (['DEBUG', 'INFO', 'WARNING', 'ERROR']):
            logging.root.setLevel(ll)
            handlers = adjust_handler('*', ll)
            logging.debug(f'adjusted log levels: {handlers}')
        else:
            logging.warning(f'unknown or invalid log_level: {ll}')
    
    # load file constants
    sentry_file = constants.sentry_file    
    expected_headers = constants.expected_headers
    
    # try to confirm created files N times before giving up
    confirm_retry = constants.confirm_retry
    
    #  wait N seconds for first try, N*retry for each subsiquent retry
    base_wait = constants.base_wait
    
    student_dirs = constants.student_dirs
        
    # get csv_file and drive_path from the command line
    try:
        csv_file = Path(config['__cmd_line']['student_export'])
    except TypeError:
        logging.info('No student export file specified on command line')
        csv_file = None

    drive_path = Path(config['main']['drive_path'])
    if not drive_path:
        logging.info('No google drive path specified on command line')
        # prompt user to enter it in GUI here
    
    # check drive path is on a shared drive 
    google_drive = gd_path(drive_path)
    try:
        google_drive.get_xattr('user.drive.id')
    except ChildProcessError as e:
        do_exit(f'The specified Google Drive "{drive_path}" is not a Shared Drive', 1)
    except FileNotFoundError as e:
        do_exit(f'The specified Google Shared Drive {drive_path}" does not exist', 1)
    
    # check if sentry file exists
    sentry_file_path = drive_path/Path(sentry_file)
    if not sentry_file_path.is_file():
        m = f'''The chosen google shared drive "{drive_path}"
does not appear to be a Cumulative Student Folder. 

The file: "{sentry_file}" is missing. 
If you are sure {drive_path} is correct, 
please contact IT Support and askfor help. 

Please screenshot or copy this entire text below and provide it to IT Support.

###############################################################################
Run the command below from the terminal of the user that submitted this ticket.
This command will create the necessary files for this script. 

Confirm that {drive_path} is the correct
Google Shared Drive for Cumulative Student folders BEFORE proceeding.
     $ touch {drive_path}/{sentry_file}'''
        logging.error(m)
        do_exit(m, 1)
 
    # read CSV into a list
    if not csv_file:
        do_exit('No CSV file specified. Exiting.', 0)
    try:
        csv_list = csv_to_list(csv_file)
    except (FileNotFoundError, OSError, IOError) as e:
        logging.error(f'could not read csv file: {csv_file}')
        logging.error(f'{e}')
        do_exit(e, 1)
    
    # map the expecdted headers to the appropriate columns
    header_map, missing_headers = map_headers(csv_list, expected_headers.keys())
    
    # error out if there are any missing headers
    if len(missing_headers) > 0:
        do_exit(f'{csv_file.name} is missing one or more headers:\n\t{missing_headers}\nprogram cannot continue', 1)
    
    # validate the csv list
    valid_rows, invalid_rows = validate_data(csv_list, expected_headers, header_map)

    # dictionary to record created directories
    directories = {'created': [], 'skipped': [], 'invalid': invalid_rows, 
                   'confirmed': [], 'failed': []}
    for row in valid_rows:
        name = row[header_map['LastFirst']]
        class_of = row[header_map['ClassOf']]
        id_number = row[header_map['Student_Number']]
        
        s_path = student_path(path=drive_path,
                              name=name, 
                              class_of=class_of, 
                              id_number=id_number)
        
        # check if there already exists a directory with the student number
        if s_path.check_similar():
            # flag those that have multiple entries
            if len(s_path.matches) > 1:
                logging.warning(f'multiple directories exist in {class_of} for student number {id_number}')
                directories['skipped'].append((s_path, 'multiple'))
            # flag those that already exist for auditing purposes
            else:
                logging.info(f'skipped {class_of}/{name} - {id_number}: folder exists')
                directories['skipped'].append((s_path, 'exists'))

                
        else:
            # create the directory and try to handle errors as needed
            try:
                s_path.mkdir(parents=True)
            except FileExistsError as e:
                logging.error(f'{s_path.student_dir_name} exists')
                directories['skipped'].append((s_path, 'exists'))
            except OSError as e:
                logging.error(f'Could not create {s_path.student_dir_name}: {e}')
                directories['skipped'].append((s_path, 'failed'))
            else:
                directories['created'].append(s_path)

# inject a bad entry to test checks at end
#     directories['created'].append(student_path(path='/Volumes/GoogleDrive/Shared drives/IT Blabla I/spam_eggs_spam',
#                                                name='Eggs, Green', class_of=1000, id_number=123456))
    
    
    # double check that drectories were created and properly synced to google drive
    for i in range(0, confirm_retry):
        if len(directories['created']) > 0:
            wait = i * base_wait
        dirs_to_check = directories['created']
        for each in dirs_to_check:
            logging.info(f'checking: {each}')
            if each.confirm():
                logging.debug(f'confirmed: {each}')
                directories['confirmed'].append(each)
                directories['created'].remove(each)
            
            # loop over the created directories N times with a longer delay each time
            # check that everything is confirmed uploaded; if it is not after Nth time, 
            # log as 'failed'
        if len(directories['created']) > 0:
            logging.info(f'sleeping for {wait} seconds and checking dirs again')
            time.sleep(wait)
        else:
            logging.info('all created directories confirmed')
            break

    
    if len(directories['created']) > 0:
        directories['failed'] = directories['created']
        directories['created'] = []
#         dirs_to_check = directories['created']
#         # migrate the created to the failed
#         for each in dirs_to_check:
#             directories['failed'].append(directories['created'].pop())
    
    # add better reporting on total attempted, created, skipped, failed, confirmed
    total = 0
    for each in directories:
        total = total + len(each)
    
    # run over skipped and report those that have multiple directories and the path
    # user should deal with these.
    skipped = len(directories['skipped'])
    failed = len(directories['failed'])
    confirmed = len(directories['confirmed'])
    invalid = len(directories['invalid'])

    print('created new:')
    for each in directories['confirmed']:
        print(each.webview_link)
    print('confirmed exist:')
    for each in directories['skipped']:
        print(each[0].webview_link)
        
    
    if update_user_config:
        try:
            logging.info(f'updating user configuration file: {user_config_path}')
            ArgConfigParse.write(config, user_config_path, create=True)
        except Exception as e:
            m = f'Error updating user configuration file: {e}'
            do_exit(m, 1)

            
    # add summary of actions and errors
    # create csv output for adding portfolio links into PS SIS
    return config
    

    # cleanup
    # handle invalid_rows -- notify user of issues


In [52]:
if __name__ == '__main__':
    f = main()

2020.07.01 13:15.07 ArgConfigParse FUNC:parse_args - INFO:ignoring unknown options: ['-f', '/Users/aaronciuffo/Library/Jupyter/runtime/kernel-f4a6f6f3-23b2-417c-8928-abe826a632be.json']
2020.07.01 13:15.07 ArgConfigParse FUNC:config_files - INFO:processing config files: [PosixPath('/Users/aaronciuffo/Documents/src/portfolioCreator/createFolders.ini'), PosixPath('/Users/aaronciuffo/.config/com.txoof.createFolders/createFolders.ini')]
2020.07.01 13:15.07 ArgConfigParse FUNC:config_files - INFO:config files not found: []
2020.07.01 13:15.07 <ipython-input-27-f3f5f0575b0c> FUNC:adjust_handler - INFO:setting <RotatingFileHandler /Users/aaronciuffo/createFolders.log (INFO)> to INFO
2020.07.01 13:15.07 <ipython-input-27-f3f5f0575b0c> FUNC:adjust_handler - INFO:setting <StreamHandler stderr (INFO)> to INFO
2020.07.01 13:15.08 <ipython-input-51-83edebd743da> FUNC:main - WARNING:multiple directories exist in 2023 for student number 505586
2020.07.01 13:15.08 <ipython-input-51-83edebd743da> FUNC:

created new:
confirmed exist:
https://drive.google.com/drive/folders/1S1va9b9RtZ8dqs3Mhm6qtTdTR5yU4ssT
https://drive.google.com/drive/folders/1S8VvqJuIsbJ379tO_IKVcAbTj93pyjgB
https://drive.google.com/drive/folders/14LSP1JHhvr1pggRqwZ2Ec8QB8VdhcayA
https://drive.google.com/drive/folders/1S7X1-sJH63Xo8Qi6YydWTOL84LAzefNB


2020.07.01 13:15.10 <ipython-input-51-83edebd743da> FUNC:main - INFO:updating user configuration file: /Users/aaronciuffo/.config/com.txoof.createFolders/createFolders.ini


https://drive.google.com/drive/folders/1S2o8u7_dFeGkMDIoK_pM2xLhZrCHq4Uw


In [ ]:
# this needs to be fixed; this does not appear to be working
# log_setup(constants.app_name)
# log_setup()

f = main()

In [53]:
f

{'main': {'drive_path': '/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)',
  'log_level': 'INFO'},
 '__cmd_line': {'student_export': './student.export.text'}}

In [34]:
sys.argv.append('-g')

In [35]:
sys.argv.append('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')
sys.argv.append('-s')
sys.argv.append('./student.export.text')

In [43]:
sys.argv.append('-l')
sys.argv.append('INFO')

In [49]:
sys.argv.pop()

'-l'

In [ ]:
# sys.argv.append('-s')
# sys.argv.append('./student.export.csv.text')
# # f = main()

In [40]:
sys.argv

['/Users/aaronciuffo/.local/share/virtualenvs/portfolioCreator-alMouNtK/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/aaronciuffo/Library/Jupyter/runtime/kernel-f4a6f6f3-23b2-417c-8928-abe826a632be.json',
 '-g',
 '/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)',
 '-s',
 './student.export.text',
 '-l',
 'DEBUG']

In [ ]:
# sys.argv.append('-g')
# sys.argv.append('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')